In [3]:
import os
import pandas as pd
from pandas import json_normalize
from google.cloud import bigquery
from google.api_core.exceptions import GoogleAPIError
from tabulate import tabulate
from dotenv import load_dotenv

In [4]:

load_dotenv()
project_id = os.getenv('GOOGLE_PROJECT_ID')
key_path = os.getenv('GOOGLE_APPLICATION_CREDENTIALS')

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = key_path
client = bigquery.Client(project=project_id)

Read the first 10 events in "ga4_obfuscated_sample_ecommerce"

In [5]:
query = """
    SELECT *
    FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
    LIMIT 10
"""

In [6]:
query_job = client.query(query)
results = query_job.result()

Convert it to dict then to DataFrame

In [8]:
rows = [dict(row) for row in results]
df = pd.DataFrame(rows)

Flattening nested fields

In [9]:
if 'event_params' in df.columns:
    event_params = json_normalize(df['event_params'])
    df = df.drop('event_params', axis=1).join(event_params)

Print the DataFrame as a table

In [10]:
print(tabulate(df, headers='keys', tablefmt='psql'))

+----+--------------+-------------------+-----------------+----------------------------+----------------------+----------------------------+---------------------------------+-----------+------------------+--------------------------------------------------------------------------------+-------------------+------------------------------+-------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-------------------------------------